# **Set hyperparameters**

In [33]:
batch_size = 200

max_seq = 60

norm         = 1e-6

seed_num = 1 # epoch no
EPOCHS  = 1

step = 20780   # 166240 #20780
warmup= 30000
limit = 10#5195    # 41560 #41560





In [2]:
b1 = 0.9
b2 = 0.98
ep =1e-9

In [3]:
dropout_att = 0.3
dropout_dff = 0.3
dropout_embd = 0.3

In [35]:
num_layers = 6
d_model = 512 # 1024
dff = 2048  # 4096
num_heads = 8 # 16

In [36]:
n=  2659723 # total sentence in txt file
buffer_size = n

# **Import Libraries**

In [37]:
#!pip install tensorflow-text
#!pip install sentencepiece

In [38]:
import logging
import time
import numpy as np
import tensorflow as tf
#import tensorflow_text as tf_text
#from tensorflow.python.platform import gfile
#from tensorflow_text import SentencepieceTokenizer, pad_model_inputs
# pylint: disable=unused-import

# **Data Pipeline**

In [39]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [40]:
!unzip /content/drive/MyDrive/nmt.zip

Archive:  /content/drive/MyDrive/nmt.zip
replace ban.model? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace corpus.train.bn? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace corpus.train.en? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace eng.model? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [41]:
import sentencepiece as spm
spe = spm.SentencePieceProcessor()
spe.load('eng.model')
# bangla tokenization
spb = spm.SentencePieceProcessor()
spb.load('ban.model')

True

In [42]:
eng_vocab= spe.vocab_size()
ban_vocab = spb.vocab_size()

# **Positional Encoding**

In [43]:
def get_angles(pos, i, d_model):
  angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
  return pos * angle_rates

In [44]:
def positional_encoding(position, d_model):
  angle_rads = get_angles(np.arange(position)[:, np.newaxis],
                          np.arange(d_model)[np.newaxis, :],
                          d_model)

  # apply sin to even indices in the array; 2i
  angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])

  # apply cos to odd indices in the array; 2i+1
  angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

  pos_encoding = angle_rads[np.newaxis, ...]

  return tf.cast(pos_encoding, dtype=tf.float32)

# **Masking**

In [45]:
def create_padding_mask(seq):
  seq = tf.cast(tf.math.equal(seq, 0), tf.float32)


  # add extra dimensions to add the padding
  # to the attention logits.
  return seq[:, tf.newaxis, tf.newaxis, :]  # (batch_size, 1, 1, seq_len)

In [46]:
def create_look_ahead_mask(size):
  mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
  return mask  # (seq_len, seq_len)

# **Scaled dot product attention**

In [47]:
def scaled_dot_product_attention(q, k, v, mask):
  """Calculate the attention weights.
  q, k, v must have matching leading dimensions.
  k, v must have matching penultimate dimension, i.e.: seq_len_k = seq_len_v.
  The mask has different shapes depending on its type(padding or look ahead)
  but it must be broadcastable for addition.

  Args:
    q: query shape == (..., seq_len_q, depth)
    k: key shape == (..., seq_len_k, depth)
    v: value shape == (..., seq_len_v, depth_v)
    mask: Float tensor with shape broadcastable
          to (..., seq_len_q, seq_len_k). Defaults to None.

  Returns:
    output, attention_weights
  """

  matmul_qk = tf.matmul(q, k, transpose_b=True)  # (..., seq_len_q, seq_len_k)
  #print('after matrxinmultiply', matmul_qk.shape )
  # scale matmul_qk
  dk = tf.cast(tf.shape(k)[-1], tf.float32)
  scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

  #print('\033[91m' + '\033[1m' + 'q*k^t , ='+'\033[0m', scaled_attention_logits,'\n')
  #print("before mask is given", scaled_attention_logits.shape)
  #print('\033[91m' + '\033[1m' + 'MASSSSKK   ='+'\033[0m', scaled_attention_logits,'\n')

  # add the mask to the scaled tensor.
  if mask is not None:
    scaled_attention_logits += (mask * -1e9)
  #print('\033[91m' + '\033[1m'+'after mask is applied, q*k^t ='+'\033[0m', scaled_attention_logits , '\n')
  #print("after mask is given", scaled_attention_logits.shape)

  # softmax is normalized on the last axis (seq_len_k) so that the scores
  # add up to 1.
  attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)  # (..., seq_len_q, seq_len_k)

  #print('\033[91m' + '\033[1m'+'after softmax ='+'\033[0m',attention_weights ,'\n')

  output = tf.matmul(attention_weights, v)  # (..., seq_len_q, depth_v)
  #print('output of the attention', output.shape)
  #print('\033[91m' + '\033[1m'+'q*k^t*v = '+'\033[0m', output ,'\n')
  return output, attention_weights

# **Multi-head attention**

In [48]:
class MultiHeadAttention(tf.keras.layers.Layer):
  def __init__(self,*, d_model, num_heads):
    super(MultiHeadAttention, self).__init__()
    self.num_heads = num_heads
    self.d_model = d_model

    assert d_model % self.num_heads == 0

    self.depth = d_model // self.num_heads

    self.wq = tf.keras.layers.Dense(d_model)
    self.wk = tf.keras.layers.Dense(d_model)
    self.wv = tf.keras.layers.Dense(d_model)

    self.dense = tf.keras.layers.Dense(d_model)

  def split_heads(self, x, batch_size):
    """Split the last dimension into (num_heads, depth).
    Transpose the result such that the shape is (batch_size, num_heads, seq_len, depth)
    """
    x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
    return tf.transpose(x, perm=[0, 2, 1, 3])

  def call(self, q,k,v, mask):
    batch_size = tf.shape(q)[0]

    q = self.wq(q)  # (batch_size, seq_len, d_model)
    k = self.wk(k)  # (batch_size, seq_len, d_model)
    v = self.wv(v)  # (batch_size, seq_len, d_model)

    q = self.split_heads(q, batch_size)  # (batch_size, num_heads, seq_len_q, depth)
    k = self.split_heads(k, batch_size)  # (batch_size, num_heads, seq_len_k, depth)
    v = self.split_heads(v, batch_size)  # (batch_size, num_heads, seq_len_v, depth)

    # scaled_attention.shape == (batch_size, num_heads, seq_len_q, depth)
    # attention_weights.shape == (batch_size, num_heads, seq_len_q, seq_len_k)
    scaled_attention, attention_weights = scaled_dot_product_attention(
        q, k, v, mask)

    scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])  # (batch_size, seq_len_q, num_heads, depth)

    concat_attention = tf.reshape(scaled_attention,
                                  (batch_size, -1, self.d_model))  # (batch_size, seq_len_q, d_model)

    output = self.dense(concat_attention)  # (batch_size, seq_len_q, d_model)

    return output, attention_weights

# **Point wise feed forward network**

In [49]:
def point_wise_feed_forward_network(d_model, dff):
  return tf.keras.Sequential([
      tf.keras.layers.Dense(dff, activation='relu'),  # (batch_size, seq_len, dff)
      tf.keras.layers.Dense(d_model)  # (batch_size, seq_len, d_model)
  ])

# **Encoder layer**

In [50]:
class EncoderLayer(tf.keras.layers.Layer):
  def __init__(self,*, d_model, num_heads, dff, rate_att=dropout_att, rate_dff=dropout_dff):
    super(EncoderLayer, self).__init__()

    self.mha = MultiHeadAttention(d_model=d_model, num_heads=num_heads)
    self.ffn = point_wise_feed_forward_network(d_model, dff)



    self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon= norm)
    self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon= norm)

    self.dropout1 = tf.keras.layers.Dropout(rate_att)
    self.dropout2 = tf.keras.layers.Dropout(rate_dff)

  def call(self, x, training, mask):

    attn_output, _ = self.mha(x, x, x, mask)  # (batch_size, input_seq_len, d_model)
    attn_output = self.dropout1(attn_output, training=training)
    out1 = self.layernorm1(x + attn_output)  # (batch_size, input_seq_len, d_model)

    ffn_output = self.ffn(out1)  # (batch_size, input_seq_len, d_model)
    #print("after feed forward neural network in encoder-", ffn_output.shape)
    ffn_output = self.dropout2(ffn_output, training=training)
    out2 = self.layernorm2(out1 + ffn_output)  # (batch_size, input_seq_len, d_model)
    #print("output of the encoder-", out2.shape )

    return out2

# **Encoder**

In [51]:
class Encoder(tf.keras.layers.Layer):
  def __init__(self,*, num_layers,  d_model, num_heads, dff,input_vocab_size,
               rate_att=dropout_att, rate_dff=dropout_dff, rate_embd=dropout_embd):
    super(Encoder, self).__init__()

    self.d_model = d_model
    self.num_layers = num_layers

    self.embedding = tf.keras.layers.Embedding(input_vocab_size, d_model)
    self.pos_encoding = positional_encoding(max_seq, self.d_model)

    self.enc_layers = [
        EncoderLayer(d_model=d_model, num_heads=num_heads, dff=dff, rate_att=rate_att, rate_dff=rate_dff)
        for _ in range(num_layers)]

    self.dropout = tf.keras.layers.Dropout(rate_embd)

  def call(self, x, training, mask):


    seq_len = tf.shape(x)[1]

    # adding embedding and position encoding.
    #print('\033[91m' + '\033[1m' +'raw token id before embedding'+'\033[0m', x, '\n')
    x = self.embedding(x)  # (batch_size, input_seq_len, d_model)
    x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
    #print('\033[91m' + '\033[1m' +'after embedding, x ='+'\033[0m', x, '\n')
    x += self.pos_encoding[:, :seq_len, :]
    #print('\033[91m' + '\033[1m' +'after adding positional encoding'+'\033[0m', x, '\n')

    x = self.dropout(x, training=training)

    for i in range(self.num_layers):
      #print('\033[91m' + '\033[1m' +'data fed to the encoder layer no [', i+1,'] applying dropout  =' +'\033[0m''\n')
      x = self.enc_layers[i](x, training, mask)
    #print('\033[91m' + '\033[1m' +'after encoder layer'+'\033[0m', x, '\n')


    return x  # (batch_size, input_seq_len, d_model)

# **Decoder layer**

In [52]:
class DecoderLayer(tf.keras.layers.Layer):
  def __init__(self,*, d_model, num_heads, dff, rate_att =dropout_att, rate_dff=dropout_dff ):
    super(DecoderLayer, self).__init__()

    self.mha1 = MultiHeadAttention(d_model=d_model, num_heads=num_heads)
    self.mha2 = MultiHeadAttention(d_model=d_model, num_heads=num_heads)

    self.ffn = point_wise_feed_forward_network(d_model, dff)

    self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon= norm)
    self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon= norm)
    self.layernorm3 = tf.keras.layers.LayerNormalization(epsilon= norm)

    self.dropout1 = tf.keras.layers.Dropout(rate_att)
    self.dropout2 = tf.keras.layers.Dropout(rate_att)
    self.dropout3 = tf.keras.layers.Dropout(rate_dff)

  def call(self, x, enc_output, training,
           look_ahead_mask, padding_mask):
    # enc_output.shape == (batch_size, input_seq_len, d_model)

    attn1, attn_weights_block1 = self.mha1(x, x, x, look_ahead_mask)  # (batch_size, target_seq_len, d_model)
    attn1 = self.dropout1(attn1, training=training)
    out1 = self.layernorm1(attn1 + x)

    attn2, attn_weights_block2 = self.mha2(
        out1, enc_output,enc_output , padding_mask)  # (batch_size, target_seq_len, d_model)
    attn2 = self.dropout2(attn2, training=training)
    out2 = self.layernorm2(attn2 + out1)  # (batch_size, target_seq_len, d_model)

    ffn_output = self.ffn(out2)  # (batch_size, target_seq_len, d_model)
    ffn_output = self.dropout3(ffn_output, training=training)
    out3 = self.layernorm3(ffn_output + out2)  # (batch_size, target_seq_len, d_model)

    return out3, attn_weights_block1, attn_weights_block2

# **Decoder**

In [53]:
class Decoder(tf.keras.layers.Layer):
  def __init__(self,*, num_layers, d_model, num_heads, dff, target_vocab_size,
               rate_att=dropout_att, rate_dff=dropout_dff, rate_embd=dropout_embd ):
    super(Decoder, self).__init__()

    self.d_model = d_model
    self.num_layers = num_layers

    self.embedding = tf.keras.layers.Embedding(target_vocab_size, d_model)
    self.pos_encoding = positional_encoding(max_seq, d_model)

    self.dec_layers = [
        DecoderLayer(d_model=d_model, num_heads=num_heads, dff=dff, rate_att=rate_att, rate_dff=rate_dff)
        for _ in range(num_layers)]
    self.dropout = tf.keras.layers.Dropout(rate_embd)

  def call(self, x, enc_output, training,
           look_ahead_mask, padding_mask):

    seq_len = tf.shape(x)[1]
    attention_weights = {}

    x = self.embedding(x)  # (batch_size, target_seq_len, d_model)
    x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
    x += self.pos_encoding[:, :seq_len, :]
    #print('\033[91m' + '\033[1m' +'after positional encoding'+'\033[0m', x, '\n')

    x = self.dropout(x, training=training)

    for i in range(self.num_layers):
      x, block1, block2 = self.dec_layers[i](x, enc_output, training,
                                             look_ahead_mask, padding_mask)

      attention_weights[f'decoder_layer{i+1}_block1'] = block1
      attention_weights[f'decoder_layer{i+1}_block2'] = block2

    #print('\033[91m' + '\033[1m' +'after decoder layer'+'\033[0m', x, '\n')

    # x.shape == (batch_size, target_seq_len, d_model)
    return x, attention_weights

# **Create the transformer model**

In [54]:
class Transformer(tf.keras.Model):
  def __init__(self,*, num_layers, d_model, num_heads, dff, input_vocab_size,
               target_vocab_size, rate_att =dropout_att, rate_dff=dropout_dff, rate_embd=dropout_embd ):
    super().__init__()
    self.encoder = Encoder(num_layers=num_layers, d_model=d_model,
                           num_heads=num_heads, dff=dff,
                           input_vocab_size=input_vocab_size, rate_att=rate_att, rate_dff=rate_dff, rate_embd=rate_embd)

    self.decoder = Decoder(num_layers=num_layers, d_model=d_model,
                           num_heads=num_heads, dff=dff,
                           target_vocab_size=target_vocab_size, rate_att=rate_att, rate_dff=rate_dff, rate_embd=rate_embd)

    self.final_layer = tf.keras.layers.Dense(target_vocab_size)

  def call(self, inputs, training):
    # Keras models prefer if you pass all your inputs in the first argument
    inp, tar = inputs
    #print('\033[91m' + '\033[1m' +'encoder input'+'\033[0m', inp, '\n')

    padding_mask, look_ahead_mask = self.create_masks(inp, tar)
    #print('\033[91m' + '\033[1m' +'padding mask=',padding_mask,'\033[0m','\n')
    #print('\033[91m' + '\033[1m' +'look ahead mask=',look_ahead_mask,'\033[0m','\n')

    enc_output = self.encoder(inp, training, padding_mask)  # (batch_size, inp_seq_len, d_model)

    #print('\033[91m' + '\033[1m' +'decoder input'+'\033[0m', tar, '\n')

    # dec_output.shape == (batch_size, tar_seq_len, d_model)
    dec_output, attention_weights = self.decoder(
        tar, enc_output, training, look_ahead_mask, padding_mask)

    final_output = self.final_layer(dec_output)  # (batch_size, tar_seq_len, target_vocab_size)

    #final_output = tf.nn.softmax([final_output],axis =-1)

    return final_output, attention_weights

  def create_masks(self, inp, tar):
    # Encoder padding mask (Used in the 2nd attention block in the decoder too.)
    padding_mask = create_padding_mask(inp)

    # Used in the 1st attention block in the decoder.
    # It is used to pad and mask future tokens in the input received by
    # the decoder.
    look_ahead_mask = create_look_ahead_mask(tf.shape(tar)[1])
    dec_target_padding_mask = create_padding_mask(tar)
    look_ahead_mask = tf.maximum(dec_target_padding_mask, look_ahead_mask)

    return padding_mask, look_ahead_mask



# **Optimizer**

In [55]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
  def __init__(self, d_model, warmup_steps=warmup):
    super(CustomSchedule, self).__init__()

    self.d_model = d_model
    self.d_model = tf.cast(self.d_model, tf.float32)

    self.warmup_steps = warmup_steps

  def __call__(self, step):
    step = tf.cast(step, dtype=tf.float32)
    arg1 = tf.math.rsqrt(step)
    arg2 = step * (self.warmup_steps ** -1.5)

    return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [56]:
learning_rate = CustomSchedule(d_model)

optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1= b1 , beta_2= b2,
                                     epsilon=  ep)

# **Loss and metrics**

In [57]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

In [58]:
def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_sum(loss_)/tf.reduce_sum(mask)


def accuracy_function(real, pred):
  accuracies = tf.equal(real, tf.argmax(pred, axis=2))

  mask = tf.math.logical_not(tf.math.equal(real, 0))
  accuracies = tf.math.logical_and(mask, accuracies)

  accuracies = tf.cast(accuracies, dtype=tf.float32)
  mask = tf.cast(mask, dtype=tf.float32)
  return tf.reduce_sum(accuracies)/tf.reduce_sum(mask)

In [59]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.Mean(name='train_accuracy')

# **Before Training and checkpointing**

In [60]:
transformer = Transformer(
    num_layers=num_layers,
    d_model=d_model,
    num_heads=num_heads,
    dff=dff,
    input_vocab_size= eng_vocab,
    target_vocab_size= ban_vocab,
    rate_att =dropout_att, rate_dff=dropout_dff, rate_embd=dropout_embd)

In [61]:
train_step_signature = [
    tf.TensorSpec(shape=(None, None), dtype=tf.int64),
    tf.TensorSpec(shape=(None, None), dtype=tf.int64),
]


@tf.function(input_signature=train_step_signature)
def train_step(inp, tar):
  tar_inp = tar[:, :-1]
  tar_real = tar[:, 1:]

  with tf.GradientTape() as tape:
    predictions, _ = transformer([inp, tar_inp],
                                 training = True)
    loss = loss_function(tar_real, predictions)

  gradients = tape.gradient(loss, transformer.trainable_variables)
  optimizer.apply_gradients(zip(gradients, transformer.trainable_variables))

  train_loss(loss)
  train_accuracy(accuracy_function(tar_real, predictions))

# **Training**

In [62]:
loaded_dataset = tf.data.Dataset.load("/content/drive/MyDrive/colabpro/data_max_seq_60")

In [63]:
for epoch in range(EPOCHS):

  start = time.time()
  train_loss.reset_states()
  train_accuracy.reset_states()
  batch = 0

  for inp, tar in loaded_dataset.cache().prefetch(tf.data.AUTOTUNE).batch(batch_size): #.batch(batch_size):
    #print('\033[91m' + '\033[1m' +'batchhhhhhhhhh =' +'\033[0m', batch ,'\033[91m' + '\033[1m' +'inppppppppppppp =' +'\033[0m', inp ,'\033[91m' + '\033[1m' +'tarrrrrrrrrr =' +'\033[0m', tar)
    batch+=1
    train_step(inp, tar)
    print( batch )
    if batch>=limit:break

  print(f'Epoch {epoch + 1} Loss {train_loss.result():.4f} Accuracy {train_accuracy.result():.4f}')
  print(f'Time taken for 1 epoch: {(time.time() - start) / 3600:.2f} hours\n')


1
2
3
4
5
6
7
8
9
10
Epoch 1 Loss 10.3836 Accuracy 0.0000
Time taken for 1 epoch: 0.02 hours



In [ ]:
tf.saved_model.save(transformer, export_dir='model')